<a href="https://colab.research.google.com/github/JuanPabloAnselmo/Data_science_Ejemplos/blob/main/NLP_Austral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers==4.30.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 38.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2


In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.4/313.4 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 7.5 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl size=55

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
from torch import nn
import torch.nn.functional as F


# Creo la clase del modelo
class RobertaModel(nn.Module):

    def __init__(self, n_classes: int = 2):
        super().__init__()
        self.roberta = AutoModel.from_pretrained("xlm-roberta-base") #Modelo transformer
        self.dropout = nn.Dropout(p=0.3) #Dropout para disminuir el overfitting
        self.linear = nn.Linear(self.roberta.config.hidden_size, self.roberta.config.hidden_size) # 1er capa linear
        self.classification = nn.Linear(self.roberta.config.hidden_size, n_classes) # 2da capa linear

    def forward(self, input_ids, attention_mask):
        #Roberta layer
        cls_output = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = torch.mean(cls_output.last_hidden_state, 1)

        # NN
        pooled_output = self.linear(pooled_output) # Primera capa
        pooled_output = F.relu(pooled_output) # Funcion de activacion relu
        pooled_output = self.dropout(pooled_output) # Dropout
        output = self.classification(pooled_output) #Segunda capa

        return output

model = RobertaModel()



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected

In [ ]:
# Cargo los pesos ya entrenados del modelo

model.load_state_dict(
    torch.load(
        f="/content/drive/MyDrive/NLP - TP - AUSTRAL/Modelo_Amazon_review.pt",
        map_location=torch.device("cpu") # Cambio modelo a cpu
    )
)

# Cargo modelo para el Tokenizer

tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [ ]:
# Creo  funcion para predecir

def predict(review_text):
    pred = {}

    encoding_review = tokenizer.encode_plus(
        review_text,
        max_length = 250, #Maximo del larogo del texto
        truncation = True, # Truncar texto
        add_special_tokens = True, #Agregar tokens especiales
        return_token_type_ids = False, # Que no devuelva los ids de esos tokens
        padding = "max_length", # Que realice padding hasta el maximo alrgo
        return_attention_mask = True, #Que devuelva la mascara de atencion
        return_tensors = 'pt' # Que los tensores que devuelve sean Pytorch
        )

    input_ids = encoding_review['input_ids']
    attention_mask = encoding_review['attention_mask']
    output = model(input_ids, attention_mask)
    _, prediction = torch.max(output, dim=1)
    if prediction == 0:
        pred["label"] = "Negativo"
        pred["score"] = f"{torch.softmax(output, dim=1)[0][0].item()*100:.2f}%"
    else:
        pred["label"] = "Positivo"
        pred["score"] = f"{torch.softmax(output, dim=1)[0][1].item()*100:.2f}%"

    return pred["label"], pred["score"]

In [ ]:
predict("La zapatilla estuvo increible")

('Positivo', '99.80%')

In [ ]:
import gradio as gr

amazon_app = gr.Interface(
    fn=predict,
    inputs=gr.Textbox(label="Introduce tu reseña aquí:", placeholder="Escribe aquí..."),
    outputs=[gr.Label(label="Predicción"), gr.Label(label="Puntaje")],
    title="Análisis de sentimientos de reseñas de productos en Español",
    description="Ingresa una reseña de algun producto y obtén una predicción sobre si su sentimiento es positivo o negativo. (Max. 250 caracteres)",
    theme="Agora",
    #layout="vertical",
    #interpretation="default",
    allow_flagging=False,
    analytics_enabled=False
)

# Ejecuta la aplicación
amazon_app.launch()

/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:584: UserWarning: Cannot load Agora. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/Agora (Request ID: Root=1-66113e93-063358cb1b256ad57b016925;3b6220b9-4b5c-49a8-a91b-af601b023d21)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")
/usr/local/lib/python3.10/dist-packages/gradio/interface.py:374: UserWarning: The `allow_flagging` parameter in `Interface` nowtakes a string value ('auto', 'manual', or 'never'), not a boolean. Setting parameter to: 'never'.
  warnings.warn(


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://fb18ac6050a84956ba.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
